#### **Import Libraries**

In [ ]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm

#### **Keyframe Extractor**

In [ ]:
class KeyFrameExtractor():

    def __init__(self, video_dir='./AIC_video', scene_dir='./SceneJSON'):
        self.video_dir = video_dir
        self.scene_dir = scene_dir
        self.parse_json_path()

    def parse_json_path(self):
        self.all_scene_paths = dict()
        for scene_part in sorted(os.listdir(self.scene_dir)):
            self.all_scene_paths[scene_part] = dict()
        for scene_part in sorted(self.all_scene_paths.keys()):
            scene_part_path = f'{self.scene_dir}/{scene_part}'
            scene_json_paths = sorted(os.listdir(scene_part_path))
            scene_json_ids = [scene_json_path.split('.')[0] for scene_json_path in scene_json_paths]
            for scene_json_id, scene_json_path in zip(scene_json_ids, scene_json_paths):
                scene_json_path_full = f'{scene_part_path}/{scene_json_path}'
                self.all_scene_paths[scene_part][scene_json_id] = scene_json_path_full

    def read_scene_json(self, scene_json_path):
        with open(scene_json_path, 'r') as f:
            video_scenes = json.load(f)
        return video_scenes

    def save_keyframes(self, keyframes, keyframe_dir):
        if not os.path.exists(keyframe_dir):
            os.makedirs(keyframe_dir)
        for (frame_id, frame) in keyframes:
            output_path = os.path.join(keyframe_dir, f'{frame_id}.jpg')
            cv2.imwrite(output_path, frame)

    def extract_keyframes(self, video_path, video_scenes, number_keyframe):
        keyframes = []
        cap = cv2.VideoCapture(video_path)
        for scene in video_scenes:
            begin_frame, end_frame = scene
            selected_frame_ids = np.linspace(begin_frame, end_frame, number_keyframe).astype(np.uint16)
            for frame_id in selected_frame_ids:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
                ret, frame = cap.read()
                if ret:
                    meta_keyframe = (frame_id, frame)
                    keyframes.append(meta_keyframe)
                else:
                    print('Error Reading Image!')
        cap.release()
        return keyframes

    def __call__(self, number_keyframe, save_dir="./Keyframe"):
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        for scene_part, scene_path_dict in self.all_scene_paths.items():
            scene_json_ids = scene_path_dict.keys()
            for scene_json_id in tqdm(scene_json_ids, desc=f'Cutting {scene_part}'):
                scene_json_full_path = scene_path_dict[scene_json_id]
                video_scenes = self.read_scene_json(scene_json_full_path)
                video_path = f"{self.video_dir}/Videos_{scene_part}/video/{scene_part}_{scene_json_id}.mp4"
                keyframes = self.extract_keyframes(video_path, video_scenes, number_keyframe)
                keyframe_save_dir = f"{save_dir}/{scene_part}/{scene_json_id}"
                self.save_keyframes(keyframes, keyframe_save_dir)

#### **Inference**

In [ ]:
# Change directory to your root path of dataset directory in your project
%cd D:/AIC2024/dataset

In [ ]:
save_dir = "./keyframe"
number_keyframe = 5
keyframe_extractor = KeyFrameExtractor()
keyframe_extractor(number_keyframe, save_dir)